In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [4]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
data = data.drop(['id'], axis=1)

In [6]:
data = data.fillna('')

In [7]:
data['content'] = data['author']+' '+ data['title']+' '+data['text']

In [8]:
data = data.drop(['title','author', 'text'], axis=1)

In [9]:
data.head()

,label,content
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...


In [10]:
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [11]:
data['content'] = data['content'].str.replace('[^\w\s]','')

In [12]:
import nltk

In [13]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
data['content']=data['content'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))

In [14]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content']=data['content'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

0    darrell lucus house dem aide: didn’t even see ...
1    daniel j. flynn flynn: hillary clinton, big wo...
2    consortiumnews.com truth might get fired truth...
3    jessica purkiss 15 civilian killed single u ai...
4    howard portnoy iranian woman jailed fictional ...
Name: content, dtype: object

In [23]:
X=data[['content']]
y=data['label']

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=45, stratify=y)

In [26]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14560, 1)
(14560,)
(6240, 1)
(6240,)


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer(analyzer='word', token_pattern=r'w{1,}', max_features=5000)
tfidf_vect.fit(data['content'])
xtrain_tfdif=tfidf_vect.transform(X_train['content'])
x_test_tfidf=tfidf_vect.transform(X_test['content'])

In [38]:
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn import metrics

pclf = PassiveAggressiveClassifier()

pclf.fit(xtrain_tfdif, y_train)

predictions = pclf.predict(x_test_tfidf)

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68      3116
           1       0.92      0.05      0.10      3124

    accuracy                           0.52      6240
   macro avg       0.72      0.52      0.39      6240
weighted avg       0.72      0.52      0.39      6240



In [39]:
print(metrics.confusion_matrix(y_test,predictions))

[[3101   15]
 [2953  171]]


In [42]:
from sklearn.neural_network import MLPClassifier

mlpclf = MLPClassifier(hidden_layer_sizes=(256,64,16),

                       activation = 'relu', 

                       solver = 'adam')

mlpclf.fit(xtrain_tfdif, y_train)

predictions = mlpclf.predict(x_test_tfidf)

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.53      0.99      0.69      3116
           1       0.94      0.12      0.22      3124

    accuracy                           0.56      6240
   macro avg       0.74      0.56      0.45      6240
weighted avg       0.74      0.56      0.45      6240



In [43]:
print(metrics.confusion_matrix(y_test,predictions))

[[3092   24]
 [2742  382]]
